In [17]:
# !git clone https://github.com/sfarrukhm/making_models_efficient.git
import os
os.chdir("/kaggle/working/making_models_efficient/intel-image-classification")

import warnings
warnings.filterwarnings("ignore")

### Create test data loader

In [4]:
from utils.data_utils import make_torch_dataset_from_image_folder
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
test_path = "/kaggle/input/intel-image-classification/seg_test/seg_test"

w,h=100,100
global_transforms = transforms.Compose([
  transforms.Resize((w, h)),
  # transforms.Grayscale(),
  transforms.ToTensor()
])


test_data=make_torch_dataset_from_image_folder(root=test_path,transform=global_transforms)
test_dataloader = DataLoader(test_data, batch_size=128, shuffle=False)


class_names = test_data.classes
class_num = len(class_names)
class_names, class_num

(['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street'], 6)

### Evaluate Models

In [14]:
import sys
sys.path.append('../')  # or whatever relative path goes up to the parent of `utils`

import torch
from models.student import StudentVisionEagle
from models.teacher import VisionEagle
from utils.kd_model_comparison import evaluate_accuracy, measure_latency, get_model_size,generate_report, count_params

student_model=StudentVisionEagle()
teacher_model=VisionEagle()

student_model.load_state_dict(torch.load("models/checkpoints/student.pth",map_location=torch.device('cpu') ))
teacher_model.load_state_dict(torch.load("models/checkpoints/teacher.pth",map_location=torch.device('cpu') ))


# Generate Report
kd_report = generate_report(
    teacher_model=teacher_model,
    student_model=student_model,
    test_loader=test_dataloader,
    device="cpu",
    teacher_path="models/checkpoints/teacher.pth",
    student_path="models/checkpoints/student.pth"
)



🚀 Running Knowledge Distillation Evaluation...



Evaluating Accuracy: 100%|██████████| 24/24 [00:30<00:00,  1.29s/it]



📊 KD Performance Report:
Metric                   Teacher        Student        Difference/Ratio
----------------------------------------------------------------------
Accuracy (%)             89.50          81.13          8.37 ↓
Latency (ms)             23088.15       5794.44        17293.71 ↓
Speedup                  -              -              3.98× ↑
Model Size (MB)          91.08          0.37           246.75× ↓
#Params (Millions)       23.83          0.09           23.73M ↓
Param Reduction (%)      -              -              99.60% ↓
----------------------------------------------------------------------
